# Assignment 2 - AIS Data
- __Group name__: SIRIUS
- __Author names__: MIMOZA MARKO(marko_mimo@bentley.edu),SNEHA ARORA(arora_sneh@bentley.edu), TAUSEEF AHMAD (ahmad_taus@bentley.edu)

# 1. Introduction

The purpose of this notebook is to put the data in a different format and generate new features. Since we generate lags of the `Average` variable we aggregate the data at the date level. The new features we create are lags, window features, datetime features, location features and status features. As an external feature we add the daily crude oil price percentage change. Our research indicated that this variable might be related to the Average value. In order to generate these features we define classes that have different functionalities. Using the transform methods of each of these classes one by one will put the data in the desired format.

# 2. Setup

In this section we import the Data Prepration notebook and the libraries.

Import the Data Preparation Notebook.

In [0]:
%run "/Courses/MA755/Groups/sirius/Project2/Data Preparation" 

Install the `future reverse_geocode plotly` package.

In [0]:
%sh /databricks/python3/bin/pip3 install future reverse_geocode plotly

Collecting future
 Downloading https://files.pythonhosted.org/packages/00/2b/8d082ddfed935f3608cc61140df6dcbf0edea1bc3ab52fb6c29ae3e81e85/future-0.16.0.tar.gz (824kB)
Collecting reverse_geocode
 Downloading https://files.pythonhosted.org/packages/1b/7f/8c77c1f192a6ecde85ef4e208386fecc69095ae3a9b07d6fa296ee504460/reverse_geocode-1.3.tar.gz (1.6MB)
Collecting plotly
 Downloading https://files.pythonhosted.org/packages/c4/0c/1b8241395302fd66b34a0fe0774ed83632afd14aa5c995b262fb7b3ac540/plotly-2.7.0.tar.gz (25.0MB)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.5/site-packages (from reverse_geocode)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.5/site-packages (from reverse_geocode)
Requirement already satisfied: decorator>=4.0.6 in /databricks/python3/lib/python3.5/site-packages (from plotly)
Collecting nbformat>=4.2 (from plotly)
 Downloading https://files.pythonhosted.org/packages/da/27/9a654d2b6cc1eaa517d1c5a4405166c7f6d72f04f6e7eea41855fe808a46/nbformat-4.4.0-py2.py3-none-any.whl (155kB)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.5/site-packages (from plotly)
Requirement already satisfied: requests in /databricks/python3/lib/python3.5/site-packages (from plotly)
Requirement already satisfied: six in /usr/lib/python3/dist-packages (from plotly)
Requirement already satisfied: ipython-genutils in /databricks/python3/lib/python3.5/site-packages (from nbformat>=4.2->plotly)
Requirement already satisfied: traitlets>=4.1 in /databricks/python3/lib/python3.5/site-packages (from nbformat>=4.2->plotly)
Collecting jupyter-core (from nbformat>=4.2->plotly)
 Downloading https://files.pythonhosted.org/packages/1d/44/065d2d7bae7bebc06f1dd70d23c36da8c50c0f08b4236716743d706762a8/jupyter_core-4.4.0-py2.py3-none-any.whl (126kB)
Collecting jsonschema!=2.5.0,>=2.4 (from nbformat>=4.2->plotly)
 Downloading https://files.pythonhosted.org/packages/77/de/47e35a97b2b05c2fadbec67d44cfcdcd09b8086951b331d82de90d2912da/jsonschema-2.6.0-py2.py3-none-any.whl
Building wheels for collected packages: future, reverse-geocode, plotly
 Running setup.py bdist_wheel for future: started
 Running setup.py bdist_wheel for future: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
 Running setup.py bdist_wheel for reverse-geocode: started
 Running setup.py bdist_wheel for reverse-geocode: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/79/6e/cd/813fa9fc04f434771c9dcc7a9eba4810479dbcfb17bc5d1819
 Running setup.py bdist_wheel for plotly: started
 Running setup.py bdist_wheel for plotly: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/0c/3e/07/4848195c61f659184ca41d5a614845a018ab2d2f2a705b9998
Successfully built future reverse-geocode plotly
Installing collected packages: future, reverse-geocode, jupyter-core, jsonschema, nbformat, plotly
Successfully installed future-0.16.0 jsonschema-2.6.0 jupyter-core-4.4.0 nbformat-4.4.0 plotly-2.7.0 reverse-geocode-1.3
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

Import the required libraries.

In [0]:
%sh pip install reverse_geocode sklearn_pandas

Collecting reverse_geocode
 Downloading https://files.pythonhosted.org/packages/1b/7f/8c77c1f192a6ecde85ef4e208386fecc69095ae3a9b07d6fa296ee504460/reverse_geocode-1.3.tar.gz (1.6MB)
Collecting sklearn_pandas
 Downloading https://files.pythonhosted.org/packages/c5/9e/42d7bcddb09a3ff52d0c60c810ba5d0fded28abbe320c85bbf7368192956/sklearn_pandas-1.6.0-py2.py3-none-any.whl
Collecting numpy (from reverse_geocode)
 Using cached https://files.pythonhosted.org/packages/43/17/cd9fa14492dbef2aaf22622db79dba087c10f125473e730cda2f2019c40b/numpy-1.14.5-cp35-cp35m-manylinux1_x86_64.whl
Collecting scipy (from reverse_geocode)
 Downloading https://files.pythonhosted.org/packages/cd/32/5196b64476bd41d596a8aba43506e2403e019c90e1a3dfc21d51b83db5a6/scipy-1.1.0-cp35-cp35m-manylinux1_x86_64.whl (33.1MB)
Collecting scikit-learn>=0.15.0 (from sklearn_pandas)
 Downloading https://files.pythonhosted.org/packages/bf/53/8c9c950a3cfaec16069df196c0b76ab05b3d1f0527f6bb97a30f4dda5240/scikit_learn-0.19.1-cp35-cp35m-manylinux1_x86_64.whl (12.2MB)
Collecting pandas>=0.11.0 (from sklearn_pandas)
 Downloading https://files.pythonhosted.org/packages/1d/cb/1cc27b57855a7f7c25965ab257e2bf251589bfe450e2cb23820b11a674a6/pandas-0.23.3-cp35-cp35m-manylinux1_x86_64.whl (8.7MB)
Collecting pytz>=2011k (from pandas>=0.11.0->sklearn_pandas)
 Downloading https://files.pythonhosted.org/packages/30/4e/27c34b62430286c6d59177a0842ed90dc789ce5d1ed740887653b898779a/pytz-2018.5-py2.py3-none-any.whl (510kB)
Collecting python-dateutil>=2.5.0 (from pandas>=0.11.0->sklearn_pandas)
 Downloading https://files.pythonhosted.org/packages/cf/f5/af2b09c957ace60dcfac112b669c45c8c97e32f94aa8b56da4c6d1682825/python_dateutil-2.7.3-py2.py3-none-any.whl (211kB)
Requirement already satisfied: six>=1.5 in /usr/lib/python3/dist-packages (from python-dateutil>=2.5.0->pandas>=0.11.0->sklearn_pandas) (1.10.0)
Building wheels for collected packages: reverse-geocode
 Running setup.py bdist_wheel for reverse-geocode: started
 Running setup.py bdist_wheel for reverse-geocode: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/79/6e/cd/813fa9fc04f434771c9dcc7a9eba4810479dbcfb17bc5d1819
Successfully built reverse-geocode
Installing collected packages: numpy, scipy, reverse-geocode, scikit-learn, pytz, python-dateutil, pandas, sklearn-pandas
Successfully installed numpy-1.14.5 pandas-0.23.3 python-dateutil-2.7.3 pytz-2018.5 reverse-geocode-1.3 scikit-learn-0.19.1 scipy-1.1.0 sklearn-pandas-1.6.0

In [0]:
from pandas import concat
from sklearn.base import BaseEstimator, TransformerMixin
import pandas  as pd
import numpy   as np
import sklearn as sk
pd.__version__, np.__version__, sk.__version__
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model,  sklearn.pipeline, \
       sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn_pandas import gen_features
from sklearn_pandas import DataFrameMapper
import reverse_geocode as rg
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import math

/dbfs/mnt/group-ma755/data/ais-test.csv
/dbfs/mnt/group-ma755/data/ais-train.csv

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-432547> in <module> () 
 11 from sklearn_pandas import gen_features 
 12 from sklearn_pandas import DataFrameMapper 
 ---> 13 import reverse_geocode as rg 
 14 from sklearn . preprocessing import StandardScaler 
 15 from sklearn . metrics import mean_squared_error , r2_score 

 /usr/local/lib/python3.5/dist-packages/reverse_geocode/__init__.py in <module> () 
 3 import collections , csv , logging , os , sys , zipfile 
 4 csv . field_size_limit ( sys . maxsize ) 
 ----> 5 from future . moves . urllib . request import urlretrieve 
 6 from scipy . spatial import cKDTree as KDTree 
 7 

 ImportError : No module named 'future'

Define the number of lags that will be used for the `Average` column, and the window width that will be used for the window statistics.

In [0]:
lag_no = 5
window_width = 2

Download the crude oil daily percentage change data for the test dataset.

In [0]:
oil_pch_test = pd.read_csv('/dbfs/FileStore/tables/DCOILWTICO_test.csv')
oil_pch_test.head()

Out[ 15 ]: 
 Date DCOILWTICO_PCH
0 1/5/2016 -2.28199
1 1/6/2016 -5.56019
2 1/7/2016 -2.00177
3 1/8/2016 -0.27035
4 1/11/2016 -5.36145

Download the crude oil daily percentage change data for the train dataset.

In [0]:
oil_pch_train =pd.read_csv('/dbfs/FileStore/tables/oil_prices.csv')
oil_pch_train.head()

Out[ 16 ]: 
 Date DCOILWTICO_PCH
0 1/4/2012 0.25253
1 1/5/2012 -1.36601
2 1/6/2012 -0.24556
3 1/9/2012 -0.24616
4 1/10/2012 0.91797

Prepare the X and y for the Train Data. Train data is saved as `X_train`. 
In `y_train` we group by the `Date` column and get the mean `Average`, then reset the index. The next step is to include rows of `y_train` starting after the `num` value and reset the index again. This is done in order for the `y_train` to match the `X_train` data after the NA values are removed.

In [0]:
num = lag_no+1
y_train = train_data.groupby(['Date'])['Average'].mean().reset_index()['Average']
y_train = y_train.loc[num:,]
y_train = y_train.reset_index()
y_train = y_train.loc[:, y_train.columns != 'index']
X_train = train_data

Scale the y for train data

In [0]:
y_train_scaled = sklearn.preprocessing.StandardScaler().fit_transform(y_train)
y_train_scaled

Out[ 27 ]: 
array([[ 1.79862832e-01],
 [-1.02662760e-01],
 [-2.12622251e-01],
 [-2.93268054e-01],
 [-3.85188353e-01],
 [-4.75119034e-01],
 [-5.13982883e-01],
 [-5.29236612e-01],
 [-5.36797156e-01],
 [-6.01393383e-01],
 [-6.23279169e-01],
 [-6.53919268e-01],
 [-6.75672412e-01],
 [-6.85487856e-01],
 [-6.99945738e-01],
 [-7.06843076e-01],
 [-7.08700052e-01],
 [-7.16658520e-01],
 [-7.21566241e-01],
 [-7.17719649e-01],
 [-7.19046060e-01],
 [-7.17984931e-01],
 [-7.19841906e-01],
 [-7.09230617e-01],
 [-7.05118742e-01],
 [-7.05781947e-01],
 [-7.08169488e-01],
 [-7.12148721e-01],
 [-7.17719649e-01],
 [-7.28861503e-01],
 [-7.13209850e-01],
 [-6.99945738e-01],
 [-6.58296425e-01],
 [-6.29380660e-01],
 [-6.21687475e-01],
 [-6.17708242e-01],
 [-6.22085399e-01],
 [-6.21820116e-01],
 [-6.23942374e-01],
 [-6.38134975e-01],
 [-6.37471769e-01],
 [-6.42246849e-01],
 [-6.46889289e-01],
 [-6.55776244e-01],
 [-6.74876566e-01],
 [-6.80182211e-01],
 [-6.87610114e-01],
 [-6.98354045e-01],
 [-7.24882269e-01],
 [-7.64939888e-01],
 [-7.64276683e-01],
 [-7.87488879e-01],
 [-8.08578818e-01],
 [-8.23965188e-01],
 [-8.18128979e-01],
 [-8.02875250e-01],
 [-7.57246703e-01],
 [-7.24882269e-01],
 [-7.01272149e-01],
 [-6.63867352e-01],
 [-6.38267616e-01],
 [-5.92241146e-01],
 [-5.63855946e-01],
 [-5.06687621e-01],
 [-4.78169780e-01],
 [-4.89709558e-01],
 [-5.22339274e-01],
 [-5.24992096e-01],
 [-5.39317338e-01],
 [-5.37195080e-01],
 [-5.41041672e-01],
 [-5.47939011e-01],
 [-5.48336934e-01],
 [-5.52581450e-01],
 [-5.64651792e-01],
 [-5.56693325e-01],
 [-5.59346147e-01],
 [-5.63590663e-01],
 [-5.58550301e-01],
 [-4.79628832e-01],
 [-3.78025732e-01],
 [-2.58648721e-01],
 [-2.46843661e-01],
 [-2.57852874e-01],
 [-2.72178116e-01],
 [-2.80534506e-01],
 [-2.80799789e-01],
 [-2.38089347e-01],
 [-2.29998238e-01],
 [-2.40211605e-01],
 [-2.48567996e-01],
 [-2.89554103e-01],
 [-3.67679724e-01],
 [-5.08544597e-01],
 [-5.63458022e-01],
 [-6.04576770e-01],
 [-6.53388704e-01],
 [-7.11352875e-01],
 [-7.44778438e-01],
 [-7.73694203e-01],
 [-8.11231640e-01],
 [-8.29934039e-01],
 [-8.46646820e-01],
 [-8.50228131e-01],
 [-8.62298473e-01],
 [-8.81398795e-01],
 [-9.24639801e-01],
 [-9.48780485e-01],
 [-9.61116110e-01],
 [-9.55545182e-01],
 [-9.46923509e-01],
 [-9.38567119e-01],
 [-9.35914296e-01],
 [-9.33128833e-01],
 [-9.24772442e-01],
 [-9.02488733e-01],
 [-8.95591395e-01],
 [-8.81398795e-01],
 [-8.58186598e-01],
 [-6.56970014e-01],
 [-5.24859455e-01],
 [-4.14369400e-01],
 [-3.70730470e-01],
 [-3.60384462e-01],
 [-3.76036115e-01],
 [-4.46733834e-01],
 [-5.57489172e-01],
 [-6.09882415e-01],
 [-6.31635560e-01],
 [-6.44103825e-01],
 [-6.78723158e-01],
 [-6.99547815e-01],
 [-7.04853460e-01],
 [-7.07373641e-01],
 [-7.28728862e-01],
 [-7.53532752e-01],
 [-7.65735735e-01],
 [-7.83377005e-01],
 [-7.96110552e-01],
 [-8.08180895e-01],
 [-8.33117426e-01],
 [-8.46646820e-01],
 [-8.34709119e-01],
 [-8.25424241e-01],
 [-8.02079403e-01],
 [-8.52483030e-01],
 [-8.71052787e-01],
 [-8.99172705e-01],
 [-9.10712483e-01],
 [-9.33261474e-01],
 [-1.00793843e+00],
 [-1.02624290e+00],
 [-1.03910909e+00],
 [-1.05157736e+00],
 [-1.05675036e+00],
 [-1.04693492e+00],
 [-1.03619099e+00],
 [-1.01974349e+00],
 [-9.74380222e-01],
 [-9.71594758e-01],
 [-9.77696250e-01],
 [-9.70135706e-01],
 [-9.63768932e-01],
 [-9.49576332e-01],
 [-9.45464457e-01],
 [-9.49443691e-01],
 [-9.51300667e-01],
 [-9.47056151e-01],
 [-9.49576332e-01],
 [-9.46392945e-01],
 [-9.32996192e-01],
 [-9.24241877e-01],
 [-9.08722866e-01],
 [-7.48625030e-01],
 [-6.32564047e-01],
 [-4.92097098e-01],
 [-4.02033775e-01],
 [-4.07869985e-01],
 [-4.22858432e-01],
 [-4.50580426e-01],
 [-4.61191716e-01],
 [-3.53089201e-01],
 [-3.02685574e-01],
 [-2.87564486e-01],
 [-2.12887533e-01],
 [-2.98427834e-02],
 [ 6.39344907e-02],
 [ 5.54454589e-02],
 [-4.33721780e-02],
 [-1.16457437e-01],
 [-4.89431051e-02],
 [ 4.27119110e-02],
 [ 1.05583803e-01],
 [ 2.89291759e-01],
 [ 3.80416211e-01],
 [ 3.49908752e-01],
 [ 4.73795562e-01],
 [ 6.32832268e-01],
 [ 9.96534228e-01],
 [

Prepare the X and y for the test data. In `y_test` we group by the `Date` column and get the mean `Average`, then reset the index. The next step is to include rows of `y_test` starting after the `num` value and reset the index again. This is done in order for the `y_test` to match the `X_test` data after the NA values are removed.

In [0]:
num = lag_no+1
y_test = test_data.groupby(['Date'])['Average'].mean().reset_index()['Average']
y_test = y_test.loc[num:,]
y_test = y_test.reset_index()
y_test = y_test.loc[:, y_test.columns != 'index']
X_test = test_data 

Scale the y for test data

In [0]:
y_test_scaled = sklearn.preprocessing.StandardScaler().fit_transform(y_test)
y_test_scaled

Out[ 95 ]: 
array([[-1.10397855e+00],
 [-1.14347936e+00],
 [-1.18298016e+00],
 [-1.23062801e+00],
 [-1.23507185e+00],
 [-1.25185969e+00],
 [-1.24099697e+00],
 [-1.21581520e+00],
 [-1.17236432e+00],
 [-1.12298831e+00],
 [-1.13928240e+00],
 [-1.15286080e+00],
 [-1.20519936e+00],
 [-1.22149345e+00],
 [-1.21630896e+00],
 [-1.21235888e+00],
 [-1.22396225e+00],
 [-1.23186241e+00],
 [-1.21186512e+00],
 [-1.22248097e+00],
 [-1.22494977e+00],
 [-1.22371537e+00],
 [-1.22865297e+00],
 [-1.24297201e+00],
 [-1.23729377e+00],
 [-1.21655584e+00],
 [-1.21803712e+00],
 [-1.22470289e+00],
 [-1.25679729e+00],
 [-1.27259761e+00],
 [-1.29012609e+00],
 [-1.29753249e+00],
 [-1.30247009e+00],
 [-1.31752978e+00],
 [-1.32740498e+00],
 [-1.33456450e+00],
 [-1.34345218e+00],
 [-1.35703058e+00],
 [-1.36073378e+00],
 [-1.36962146e+00],
 [-1.34468658e+00],
 [-1.35308050e+00],
 [-1.36147442e+00],
 [-1.37283090e+00],
 [-1.37628722e+00],
 [-1.36320258e+00],
 [-1.40690035e+00],
 [-1.41899747e+00],
 [-1.40640659e+00],
 [-1.40764099e+00],
 [-1.41677555e+00],
 [-1.41405987e+00],
 [-1.39505011e+00],
 [-1.39085315e+00],
 [-1.37801538e+00],
 [-1.28741041e+00],
 [-1.16520480e+00],
 [-1.04201166e+00],
 [-9.61035014e-01],
 [-9.14868449e-01],
 [-7.89947154e-01],
 [-6.52681858e-01],
 [-5.43067125e-01],
 [-5.42079604e-01],
 [-5.47510965e-01],
 [-3.82348225e-01],
 [-1.62131239e-01],
 [ 1.55022026e-03],
 [ 7.19110286e-02],
 [ 3.03150044e-03],
 [-2.31377827e-02],
 [ 1.19311994e-01],
 [ 1.34124796e-01],
 [ 2.60774251e-01],
 [ 3.54835542e-01],
 [ 3.25209939e-01],
 [ 2.85956014e-01],
 [ 1.42271837e-01],
 [-7.76982692e-02],
 [-1.37690116e-01],
 [-1.74969001e-01],
 [-2.76436693e-01],
 [-4.41105672e-01],
 [-5.46029685e-01],
 [-5.38376404e-01],
 [-3.57166462e-01],
 [-2.45082929e-01],
 [ 1.43879818e-02],
 [-9.18580034e-04],
 [-8.75734703e-02],
 [-1.85584842e-01],
 [-1.94225643e-01],
 [-2.48539249e-01],
 [-3.58400863e-01],
 [-3.84570146e-01],
 [-3.42353661e-01],
 [-2.94952695e-01],
 [-2.94212055e-01],
 [-3.19640698e-01],
 [-2.41626609e-01],
 [-2.32985808e-01],
 [-2.37923408e-01],
 [-1.67562600e-01],
 [-1.40405797e-01],
 [-1.21396034e-01],
 [-1.13248993e-01],
 [-9.96705918e-02],
 [-1.38924516e-01],
 [-1.96200683e-01],
 [-2.96187095e-01],
 [-3.50006942e-01],
 [-4.01111108e-01],
 [-4.01357988e-01],
 [-3.65313503e-01],
 [-3.16184378e-01],
 [-3.06556056e-01],
 [-2.80880533e-01],
 [-2.34220208e-01],
 [-1.53737318e-01],
 [-9.37454711e-02],
 [-5.99229071e-02],
 [-8.55984301e-02],
 [-1.15470914e-01],
 [-1.34480676e-01],
 [-1.44108997e-01],
 [-1.59909319e-01],
 [-1.70031400e-01],
 [-8.97953906e-02],
 [-5.02945859e-02],
 [-4.56038653e-02],
 [-6.21448273e-02],
 [-1.17199074e-01],
 [-2.15704206e-01],
 [-2.71745972e-01],
 [-3.04827896e-01],
 [-3.48772541e-01],
 [-4.13208229e-01],
 [-4.96159919e-01],
 [-5.43807765e-01],
 [-5.66026967e-01],
 [-5.87258650e-01],
 [-5.83555449e-01],
 [-5.83555449e-01],
 [-5.85036729e-01],
 [-5.64545687e-01],
 [-5.51954806e-01],
 [-5.86518010e-01],
 [-5.50473525e-01],
 [-4.79372077e-01],
 [-3.69510464e-01],
 [-3.14949977e-01],
 [-3.02605976e-01],
 [-3.74941825e-01],
 [-4.37155592e-01],
 [-4.65793675e-01],
 [-4.56165354e-01],
 [-4.25799111e-01],
 [-3.14209337e-01],
 [-2.23851247e-01],
 [-2.00397644e-01],
 [-2.28295087e-01],
 [-2.47551729e-01],
 [-2.22616846e-01],
 [-1.33740036e-01],
 [-6.75761880e-02],
 [ 1.37087356e-01],
 [ 4.13592989e-01],
 [ 6.01715572e-01],
 [ 7.19971106e-01],
 [ 7.38733988e-01],
 [ 7.09355264e-01],
 [ 3.76560985e-01],
 [ 4.65437795e-01],
 [ 7.89097514e-01],
 [ 1.09473499e+00],
 [ 1.33815870e+00],
 [ 1.63762417e+00],
 [ 1.86845700e+00],
 [ 1.84401588e+00],
 [ 1.74156067e+00],
 [ 1.68058130e+00],
 [ 1.50998720e+00],
 [ 1.31742078e+00],
 [ 1.22879085e+00],
 [ 1.15843004e+00],
 [ 1.14732044e+00],
 [ 1.24434429e+00],
 [ 1.65663394e+00],
 [ 1.70082546e+00],
 [ 1.68823458e+00],
 [ 1.66724978e+00],
 [ 1.49369312e+00],
 [ 1.24360365e+00],
 [ 1.23718477e+00],
 [ 1.21249676e+00],
 [ 1.10362267e+00],
 [ 8.66864723e-01],
 [ 5.90112210e-01],
 [ 4.75559877e-01],
 [

# 3. Define Classes of Different Functionality

In this section we define all the classes and explain their functionality through creating objects and giving information about their output.

## 3.1 `AddLocationAndAggregate` Class

Define the `AddLocationAndAggregate` class. This class will take the train data as input and return a new data aggregated in date level with new location and status features.

This class will perform the following steps:
- Add location columns: `City`, `Country` and `country_code`.
- Binarize the `Status` and the `Country` variables.
- Group by the data by `Date` and `Average` and aggregate the other variables (Binarized `Status` and `Country` variables) by the sum.
- Group by `Date` and aggregate the `dwt` variable by the mean.
- Concatenate the last two steps and return the concatenated dataframe as output.

In [0]:
class AddLocationAndAggregate(BaseEstimator, TransformerMixin):
  
  def __init__(self):
    self
    
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.prepare(X)
    return self.transformed_
  
  def prepare(self, X):
    X.loc[X['Longitude'] > 180, 'Longitude'] = 180
    X['location'] = X[['Latitude', 'Longitude']].apply(tuple, axis=1)
    results = pd.DataFrame(rg.search(list(X['location']))) # rg.search(loc) gives list of dictionaries i.e. {'city': 'Hoek van Holland', 'country': 'Netherlands', 'country_code': 'NL'} we store it as df with 3 columns city, country and country_code
    X['City']=results.city # store the city column 
    X['Country'] = results.country  # store the country column 
    X['country_code'] = results.country_code  # store the country_code column 
    dwt_df = pd.DataFrame(X.groupby(['Date'])['dwt'].mean().reset_index()['dwt']) # get dwt mean value for each date
    feature_list = ['Status','Country']
    binarizer_mapper=  DataFrameMapper(gen_features(columns=feature_list,
                                     classes=[{'class': sklearn.preprocessing.LabelBinarizer}]
                                   ),df_out=True)                     
    bin_df= binarizer_mapper.fit_transform(X)
    
    list1= ['Date', 'Average']
    date_average = X.loc[: , list1] # get only the Date and Average from the input data to concatenate in next line with new variables of binarizer_mapper
    new_X = concat([date_average, bin_df], axis=1) # at this point the data contains only Date, Average, Status and Country binarized variables
    list_x = [item for item in list(new_X.columns) if item not in list1] #exclude Date and Average from list_x because in next line we sum all the other cols
    X = new_X.groupby(['Date', 'Average'])[list_x].sum().reset_index()  # group by Date and Average the other cols will be aggregated with sum function
    X = concat([X, dwt_df], axis=1)   # concat X and the dwt_df
    return X

Out[ 8 ]: 
 Latitude Longitude Status vessel_id Date dwt Average
0 37.24127 -76.10663 Port 48 2017-01-31 173900 8515
1 54.39217 18.71163 Port 138 2017-01-31 170000 8515
2 36.97648 -1.86980 Port 1726 2017-01-31 180200 8515
3 17.30403 73.20397 Port 2253 2017-01-31 172000 8515
4 -20.09275 118.60607 Port 2276 2017-01-31 170400 8515
5 31.21118 122.57270 Port 2381 2017-01-31 174300 8515
6 -23.00639 -44.06941 Port 2515 2017-01-31 177700 8515
7 48.71947 -123.54226 Port 3203 2017-01-31 177000 8515
8 36.68098 36.15170 Port 3299 2017-01-31 178000 8515
9 -21.81740 -40.81880 Port 3387 2017-01-31 180000 8515

Generate an object of `AddLocationAndAggregate` class and check the output of its transform method.

In [0]:
prepare_data = AddLocationAndAggregate()
aggregated_data = prepare_data.fit_transform(X_train)
aggregated_data.shape, aggregated_data

Out[ 97 ]: 
((999, 181),
 Date Average Status_Ballast Status_Laden Status_Port \
 0 2012-01-03 24795 197 285 217 
 1 2012-01-04 22697 208 290 195 
 2 2012-01-05 18729 204 279 191 
 3 2012-01-06 16424 201 266 193 
 4 2012-01-09 15498 203 276 209 
 5 2012-01-10 14368 216 274 194 
 6 2012-01-11 13129 206 266 199 
 7 2012-01-12 10999 213 257 201 
 8 2012-01-13 10170 203 256 234 
 9 2012-01-16 9562 228 241 244 
 10 2012-01-17 8869 225 238 241 
 11 2012-01-18 8191 221 240 258 
 12 2012-01-19 7898 225 256 250 
 13 2012-01-20 7783 225 255 242 
 14 2012-01-23 7726 229 266 227 
 15 2012-01-24 7239 222 266 244 
 16 2012-01-25 7074 232 268 246 
 17 2012-01-26 6843 216 262 248 
 18 2012-01-27 6679 233 264 230 
 19 2012-01-30 6605 216 252 234 
 20 2012-01-31 6496 209 254 239 
 21 2012-02-01 6444 216 248 250 
 22 2012-02-02 6430 214 241 270 
 23 2012-02-03 6370 211 244 270 
 24 2012-02-06 6333 199 253 262 
 25 2012-02-07 6362 201 249 277 
 26 2012-02-08 6352 209 245 281 
 27 2012-02-09 6360 217 235 271 
 28 2012-02-10 6346 211 246 253 
 29 2012-02-13 6426 204 255 259 
 .. ... ... ... ... ... 
 969 2015-11-13 6280 341 414 354 
 970 2015-11-16 6135 340 421 359 
 971 2015-11-17 5968 337 434 347 
 972 2015-11-18 5597 335 439 349 
 973 2015-11-19 5280 337 410 361 
 974 2015-11-20 5211 323 429 352 
 975 2015-11-23 5967 332 421 353 
 976 2015-11-24 6409 325 429 364 
 977 2015-11-25 7062 328 424 369 
 978 2015-11-26 7682 309 421 381 
 979 2015-11-27 8396 310 438 383 
 980 2015-11-30 8486 325 432 356 
 981 2015-12-01 8977 322 433 365 
 982 2015-12-02 8715 322 450 362 
 983 2015-12-03 8189 310 432 374 
 984 2015-12-04 7863 316 416 384 
 985 2015-12-07 7522 334 418 378 
 986 2015-12-08 7609 341 409 374 
 987 2015-12-09 7486 325 411 379 
 988 2015-12-10 7181 324 405 391 
 989 2015-12-11 6799 300 413 404 
 990 2015-12-14 6341 315 428 394 
 991 2015-12-15 5514 315 432 356 
 992 2015-12-16 5044 300 441 366 
 993 2015-12-17 5008 307 439 376 
 994 2015-12-18 5190 313 427 375 
 995 2015-12-21 5156 310 441 342 
 996 2015-12-22 4958 302 440 364 
 997 2015-12-23 4897 302 447 362 
 998 2015-12-24 4965 291 455 372 
 
 Country_Aland Islands Country_Albania Country_Algeria \
 0 0 0 0 
 1 0 0 1 
 2 0 0 2 
 3 0 0 0 
 4 0 0 1 
 5 0 0 0 
 6 0 0 2 
 7 0 0 5 
 8 0 0 3 
 9 0 0 2 
 10 0 0 1 
 11 0 0 2 
 12 0 0 0 
 13 0 0 2 
 14 0 0 0 
 15 0 0 2 
 16 0 0 3 
 17 0 0 2 
 18 0 0 0 
 19 0 0 1 
 20 0 0 1 
 21 0 0 3 
 22 0 0 1 
 23 0 0 0 
 24 0 0 1 
 25 0 0 0 
 26 0 0 0 
 27 0 0 2 
 28 0 0 3 
 29 0 0 2 
 .. ... ... ... 
 969 0 0 3 
 970 0 0 3 
 971 0 0 3 
 972 0 0 1 
 973 0 0 1 
 974 0 0 0 
 975 0 0 2 
 976 0 0 0 
 977 0 0 2 
 978 0 0 2 
 979 0 0 1 
 980 0 0 0 
 981 0 0 2 
 982 0 0 5 
 983 0 0 3 
 984 0 0 0 
 985 0 0 2 
 986 0 0 2 
 987 0 0 0 
 988 0 0 1 
 989 0 0 3 
 990 0 0 1 
 991 0 0 3 
 992 0 0 2 
 993 0 0 0 
 994 0 0 1 
 995 0 0 6 
 996 0 0 8 
 997 0 0 2 
 998 0 0 1 
 
 Country_American Samoa Country_Angola ... Country_Uruguay \
 0 0 0 ... 0 
 1 0 0 ... 0 
 2 0 0 ... 0 
 3 0 1 ... 0 
 4 0 2 ... 0 
 5 0 0 ... 0 
 6 0 4 ... 0 
 7 0 1 ... 0 
 8 0 1 ... 0 
 9 0 0 ... 0 
 10 0 2 ... 0 
 11 0 2 ... 0 
 12 0 3 ... 0 
 13 0 3 ... 0 
 14 0 1 ... 0 
 15 0 0 ... 0 
 16 0 2 ... 0 
 17 0 0 ... 0 
 18 0 1 ... 0 
 19 0 0 ... 0 
 20 0 1 ... 0 
 21 0 1 ... 0 
 22 0 0 ... 0 
 23 0 0 ... 0 
 24 0 0 ... 0 
 25 0 2 ... 0 
 26 0 1 ... 0 
 27 0 0 ... 0 
 28 0 1 ... 0 
 29 0 0 ... 0 
 .. ... ... ... ... 
 969 0 1 ... 0 
 970 0 2 ... 0 
 971 1 0 ... 0 
 972 1 3 ... 0 
 973 2 2 ... 0 
 974 1 0 ... 0 
 975 0 3 ... 2 
 976 0 0 ... 1 
 977 0 2 ... 0 
 978 0 5 ... 0 
 979 0 1 ... 1 
 980 0 2 ... 0 
 981 0 2 ... 0 
 982 0 1 ... 0 
 983 0 1 ... 0 
 984 0 0 ... 0 
 985 0 1 ... 0 
 986 0 1 ... 0 
 987 0 1 ... 1 
 988 0 1 ... 0 
 989 0 2 ... 0 
 990 0 2 ... 0 
 991 0 1 ... 0 
 992 0 2 ... 1 
 993 0 1 ... 0 
 994 0 3 ... 0 
 995 0 0 ... 0 
 996 0 0 ... 0 
 997 0 1 ... 0 
 998 0 0 ... 0 
 
 Country_Vanuatu Country_Venezuela Country_Vietnam \
 0 0 0 9 
 1 0 0 11 
 2 0 0 4 
 3 0 0 6 
 4 0 0 12 
 5 0 0 13 
 6 0 0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983171 entries, 0 to 983170
Data columns (total 7 columns):
Latitude 983171 non-null float64
Longitude 983171 non-null float64
Status 983171 non-null object
vessel_id 983171 non-null int64
Date 983171 non-null datetime64[ns]
dwt 983171 non-null int64
Average 983171 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 52.5+ MB

The output of the transform method of `AddLocationAndAggregate` class is a dataframe with 999 rows and 181 columns. Each row corresponds to a unique date of the train data. The columns are the date, the average value, three columns for each status, 175 columns for each country and the dead weight. The average value corresponds to each date while the three variables related to status indicate how many ships are in that date in each status level. The 175 country variables indicate how many ships are in that date in each particular country. The dead weight is the mean of the dead weight of all the ships for that date.

## 3.2 `ExternalFeatures` Class

Define the `ExternalFeatures` class. This class will use as input the output of `AddLocationAndAggregate` class transform method and will return a dataframe that is the same as the input but with an extra column, the crude oil price change. 

The class performs the following steps: 
- Check if the input data is train or test by evaluating the number of rows of the input.
- If it is train performs a left join by date of the input with the oil price corresponding to train data. Then, it fills the NA values using the forward fill method i.e. replacing the current value with the value ocurring before.
- If it is test performs a left join by date of the input with the oil price corresponding to test data. Then, it fills the NA values using the forward fill method i.e. replacing the current value with the value ocurring before.

In [0]:
class ExternalFeatures(BaseEstimator, TransformerMixin):
  
  def __init__(self):
    self
    
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    rows = X.shape[0]
    if rows == 999:
      self.transformed_ = self.add_oil_train(X)
    else: 
      self.transformed_ = self.add_oil_test(X)
    return self.transformed_
    
  
  def add_oil_train(self, X):
    oil_pch_train['Date'] = pd.to_datetime(oil_pch_train['Date'],format=None)
    result = pd.merge(X, oil_pch_train, on = ['Date'], how ='left')
    temp = result.replace('.', np.NaN)
    result = temp.fillna(method='ffill') #forward fill missing values
    result['DCOILWTICO_PCH'][0] = 0 #custom value for 2012-01-03 date since this value is not available in the oil data
    return result
  
  def add_oil_test(self, X):
    oil_pch_test['Date'] = pd.to_datetime(oil_pch_test['Date'],format=None)
    result = pd.merge(X, oil_pch_test, on = ['Date'], how ='left')
    temp = result.replace('.', np.NaN)
    result = temp.fillna(method='ffill') #forward fill missing values
    result['DCOILWTICO_PCH'][0] = 0 #custom value for 2012-01-03 date since this value is not available in the oil data
    return result

Generate an object of `ExternalFeatures` class and check the output of its transform method.

In [0]:
add_ext_features = ExternalFeatures()
add_to_train = add_ext_features.fit_transform(aggregated_data)
add_to_train.shape, add_to_train

/tmp/1525746643086-0/PythonShell.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import matplotlib as mpl
 Out[ 99 ]: 
((999, 182),
 Date Average Status_Ballast Status_Laden Status_Port \
 0 2012-01-03 24795 197 285 217 
 1 2012-01-04 22697 208 290 195 
 2 2012-01-05 18729 204 279 191 
 3 2012-01-06 16424 201 266 193 
 4 2012-01-09 15498 203 276 209 
 5 2012-01-10 14368 216 274 194 
 6 2012-01-11 13129 206 266 199 
 7 2012-01-12 10999 213 257 201 
 8 2012-01-13 10170 203 256 234 
 9 2012-01-16 9562 228 241 244 
 10 2012-01-17 8869 225 238 241 
 11 2012-01-18 8191 221 240 258 
 12 2012-01-19 7898 225 256 250 
 13 2012-01-20 7783 225 255 242 
 14 2012-01-23 7726 229 266 227 
 15 2012-01-24 7239 222 266 244 
 16 2012-01-25 7074 232 268 246 
 17 2012-01-26 6843 216 262 248 
 18 2012-01-27 6679 233 264 230 
 19 2012-01-30 6605 216 252 234 
 20 2012-01-31 6496 209 254 239 
 21 2012-02-01 6444 216 248 250 
 22 2012-02-02 6430 214 241 270 
 23 2012-02-03 6370 211 244 270 
 24 2012-02-06 6333 199 253 262 
 25 2012-02-07 6362 201 249 277 
 26 2012-02-08 6352 209 245 281 
 27 2012-02-09 6360 217 235 271 
 28 2012-02-10 6346 211 246 253 
 29 2012-02-13 6426 204 255 259 
 .. ... ... ... ... ... 
 969 2015-11-13 6280 341 414 354 
 970 2015-11-16 6135 340 421 359 
 971 2015-11-17 5968 337 434 347 
 972 2015-11-18 5597 335 439 349 
 973 2015-11-19 5280 337 410 361 
 974 2015-11-20 5211 323 429 352 
 975 2015-11-23 5967 332 421 353 
 976 2015-11-24 6409 325 429 364 
 977 2015-11-25 7062 328 424 369 
 978 2015-11-26 7682 309 421 381 
 979 2015-11-27 8396 310 438 383 
 980 2015-11-30 8486 325 432 356 
 981 2015-12-01 8977 322 433 365 
 982 2015-12-02 8715 322 450 362 
 983 2015-12-03 8189 310 432 374 
 984 2015-12-04 7863 316 416 384 
 985 2015-12-07 7522 334 418 378 
 986 2015-12-08 7609 341 409 374 
 987 2015-12-09 7486 325 411 379 
 988 2015-12-10 7181 324 405 391 
 989 2015-12-11 6799 300 413 404 
 990 2015-12-14 6341 315 428 394 
 991 2015-12-15 5514 315 432 356 
 992 2015-12-16 5044 300 441 366 
 993 2015-12-17 5008 307 439 376 
 994 2015-12-18 5190 313 427 375 
 995 2015-12-21 5156 310 441 342 
 996 2015-12-22 4958 302 440 364 
 997 2015-12-23 4897 302 447 362 
 998 2015-12-24 4965 291 455 372 
 
 Country_Aland Islands Country_Albania Country_Algeria \
 0 0 0 0 
 1 0 0 1 
 2 0 0 2 
 3 0 0 0 
 4 0 0 1 
 5 0 0 0 
 6 0 0 2 
 7 0 0 5 
 8 0 0 3 
 9 0 0 2 
 10 0 0 1 
 11 0 0 2 
 12 0 0 0 
 13 0 0 2 
 14 0 0 0 
 15 0 0 2 
 16 0 0 3 
 17 0 0 2 
 18 0 0 0 
 19 0 0 1 
 20 0 0 1 
 21 0 0 3 
 22 0 0 1 
 23 0 0 0 
 24 0 0 1 
 25 0 0 0 
 26 0 0 0 
 27 0 0 2 
 28 0 0 3 
 29 0 0 2 
 .. ... ... ... 
 969 0 0 3 
 970 0 0 3 
 971 0 0 3 
 972 0 0 1 
 973 0 0 1 
 974 0 0 0 
 975 0 0 2 
 976 0 0 0 
 977 0 0 2 
 978 0 0 2 
 979 0 0 1 
 980 0 0 0 
 981 0 0 2 
 982 0 0 5 
 983 0 0 3 
 984 0 0 0 
 985 0 0 2 
 986 0 0 2 
 987 0 0 0 
 988 0 0 1 
 989 0 0 3 
 990 0 0 1 
 991 0 0 3 
 992 0 0 2 
 993 0 0 0 
 994 0 0 1 
 995 0 0 6 
 996 0 0 8 
 997 0 0 2 
 998 0 0 1 
 
 Country_American Samoa Country_Angola ... Country_Vanuatu \
 0 0 0 ... 0 
 1 0 0 ... 0 
 2 0 0 ... 0 
 3 0 1 ... 0 
 4 0 2 ... 0 
 5 0 0 ... 0 
 6 0 4 ... 0 
 7 0 1 ... 0 
 8 0 1 ... 0 
 9 0 0 ... 0 
 10 0 2 ... 0 
 11 0 2 ... 0 
 12 0 3 ... 0 
 13 0 3 ... 0 
 14 0 1 ... 0 
 15 0 0 ... 0 
 16 0 2 ... 0 
 17 0 0 ... 0 
 18 0 1 ... 0 
 19 0 0 ... 0 
 20 0 1 ... 0 
 21 0 1 ... 0 
 22 0 0 ... 0 
 23 0 0 ... 0 
 24 0 0 ... 0 
 25 0 2 ... 0 
 26 0 1 ... 0 
 27 0 0 ... 0 
 28 0 1 ... 0 
 29 0 0 ... 0 
 .. ... ... ... ... 
 969 0 1 ... 1 
 970 0 2 ... 0 
 971 1 0 ... 0 
 972 1 3 ... 0 
 973 2 2 ... 0 
 974 1 0 ... 0 
 975 0 3 ... 0 
 976 0 0 ... 0 
 977 0 2 ... 0 
 978 0 5 ... 1 
 979 0 1 ... 1 
 980 0 2 ... 0 
 981 0 2 ... 0 
 982 0 1 ... 0 
 983 0 1 ... 0 
 984 0 0 ... 0 
 985 0 1 ... 0 
 986 0 1 ... 0 
 987 0 1 ... 0 
 988 0 1 ... 0 
 989

The output of the `ExternalFeatures` transform method is a new dataframe that contains all of the columns of the `AddLocationAndAggregate` and the column with the crude oil price percentage change. This new dataframe has 999 rows and 182 columns.

## 3.3 `Lags` Class

Define the `Lags` class. This class will be used in feature union. It will take as input a column and generate as many lags as specified in the `lag_no` parameter. The output of this class transform method is a dataframe containing the generated lags. 
In order to generate the lags it uses the `shift()` function in a `for loop`. This function shifts the index by the desired number of periods which in this case is the `i` value inside the loop.

In [0]:
class Lags(BaseEstimator, TransformerMixin):
  
  def __init__(self, lag_no):
    self.lag_no = lag_no
    
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.create_lags(X)
    return self.transformed_
  
  def create_lags(self, X):
    num = self.lag_no+1
    df = pd.DataFrame()
    d = pd.DataFrame(X)
    for i in range(1,num):
      df = concat([df, d.shift(i)], axis=1)
    return df

Create a `lag_mapper` that uses the `Lags` class to generate lags from the `Average` column, and check the output of the transform method. The input is the aggregated data in `Date` level.

In [0]:
lag_mapper =  DataFrameMapper([('Average', Lags(lag_no))],                            
                                 df_out=True)
lag_mapper.fit_transform(add_to_train)

Out[ 101 ]: 
 Average_0 Average_1 Average_2 Average_3 Average_4
0 NaN NaN NaN NaN NaN
1 24795.0 NaN NaN NaN NaN
2 22697.0 24795.0 NaN NaN NaN
3 18729.0 22697.0 24795.0 NaN NaN
4 16424.0 18729.0 22697.0 24795.0 NaN
5 15498.0 16424.0 18729.0 22697.0 24795.0
6 14368.0 15498.0 16424.0 18729.0 22697.0
7 13129.0 14368.0 15498.0 16424.0 18729.0
8 10999.0 13129.0 14368.0 15498.0 16424.0
9 10170.0 10999.0 13129.0 14368.0 15498.0
10 9562.0 10170.0 10999.0 13129.0 14368.0
11 8869.0 9562.0 10170.0 10999.0 13129.0
12 8191.0 8869.0 9562.0 10170.0 10999.0
13 7898.0 8191.0 8869.0 9562.0 10170.0
14 7783.0 7898.0 8191.0 8869.0 9562.0
15 7726.0 7783.0 7898.0 8191.0 8869.0
16 7239.0 7726.0 7783.0 7898.0 8191.0
17 7074.0 7239.0 7726.0 7783.0 7898.0
18 6843.0 7074.0 7239.0 7726.0 7783.0
19 6679.0 6843.0 7074.0 7239.0 7726.0
20 6605.0 6679.0 6843.0 7074.0 7239.0
21 6496.0 6605.0 6679.0 6843.0 7074.0
22 6444.0 6496.0 6605.0 6679.0 6843.0
23 6430.0 6444.0 6496.0 6605.0 6679.0
24 6370.0 6430.0 6444.0 6496.0 6605.0
25 6333.0 6370.0 6430.0 6444.0 6496.0
26 6362.0 6333.0 6370.0 6430.0 6444.0
27 6352.0 6362.0 6333.0 6370.0 6430.0
28 6360.0 6352.0 6362.0 6333.0 6370.0
29 6346.0 6360.0 6352.0 6362.0 6333.0
.. ... ... ... ... ...
969 6655.0 7062.0 7543.0 7518.0 7358.0
970 6280.0 6655.0 7062.0 7543.0 7518.0
971 6135.0 6280.0 6655.0 7062.0 7543.0
972 5968.0 6135.0 6280.0 6655.0 7062.0
973 5597.0 5968.0 6135.0 6280.0 6655.0
974 5280.0 5597.0 5968.0 6135.0 6280.0
975 5211.0 5280.0 5597.0 5968.0 6135.0
976 5967.0 5211.0 5280.0 5597.0 5968.0
977 6409.0 5967.0 5211.0 5280.0 5597.0
978 7062.0 6409.0 5967.0 5211.0 5280.0
979 7682.0 7062.0 6409.0 5967.0 5211.0
980 8396.0 7682.0 7062.0 6409.0 5967.0
981 8486.0 8396.0 7682.0 7062.0 6409.0
982 8977.0 8486.0 8396.0 7682.0 7062.0
983 8715.0 8977.0 8486.0 8396.0 7682.0
984 8189.0 8715.0 8977.0 8486.0 8396.0
985 7863.0 8189.0 8715.0 8977.0 8486.0
986 7522.0 7863.0 8189.0 8715.0 8977.0
987 7609.0 7522.0 7863.0 8189.0 8715.0
988 7486.0 7609.0 7522.0 7863.0 8189.0
989 7181.0 7486.0 7609.0 7522.0 7863.0
990 6799.0 7181.0 7486.0 7609.0 7522.0
991 6341.0 6799.0 7181.0 7486.0 7609.0
992 5514.0 6341.0 6799.0 7181.0 7486.0
993 5044.0 5514.0 6341.0 6799.0 7181.0
994 5008.0 5044.0 5514.0 6341.0 6799.0
995 5190.0 5008.0 5044.0 5514.0 6341.0
996 5156.0 5190.0 5008.0 5044.0 5514.0
997 4958.0 5156.0 5190.0 5008.0 5044.0
998 4897.0 4958.0 5156.0 5190.0 5008.0

[999 rows x 5 columns]

The output above is a dataframe containing the lags generated from the `Average` variable. There are five lags generated. There are NA values because the past values at those points do not exist.

## 3.4 `DatetimeGenerator` Class

Define the `DatetimeGenerator` class. This class will be used in feature union. It will take as input a `date` variable and the output of the transform method is a dataframe containing the `month`, `day` and `year` columns.

In [0]:
class DatetimeGenerator(BaseEstimator, TransformerMixin):
  
  def __init__(self):
    self
    
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.datetime_features(X)
    return self.transformed_
  
  def datetime_features(self, X):
    df=pd.DataFrame()
    a = pd.to_datetime(X)
    df['month'] = [month.month for month in a]
    df['day'] = [day.day for day in a]
    df['year'] = [year.year for year in a]
    return df

Create a `datetime_mapper` that uses the `DatetimeGenerator` class to generate month, day and year columns from the `Date` column, and check the output of the transform method. The output is the aggregated data at `Date` level.

In [0]:
date_list=['Date']
datetime_mapper = DataFrameMapper(gen_features(columns=date_list, 
                                             classes=[{'class': DatetimeGenerator}
                                                     ]), df_out=True)
datetime_mapper.fit_transform(add_to_train)

Out[ 103 ]: 
 Date_0 Date_1 Date_2
0 1 3 2012
1 1 4 2012
2 1 5 2012
3 1 6 2012
4 1 9 2012
5 1 10 2012
6 1 11 2012
7 1 12 2012
8 1 13 2012
9 1 16 2012
10 1 17 2012
11 1 18 2012
12 1 19 2012
13 1 20 2012
14 1 23 2012
15 1 24 2012
16 1 25 2012
17 1 26 2012
18 1 27 2012
19 1 30 2012
20 1 31 2012
21 2 1 2012
22 2 2 2012
23 2 3 2012
24 2 6 2012
25 2 7 2012
26 2 8 2012
27 2 9 2012
28 2 10 2012
29 2 13 2012
.. ... ... ...
969 11 13 2015
970 11 16 2015
971 11 17 2015
972 11 18 2015
973 11 19 2015
974 11 20 2015
975 11 23 2015
976 11 24 2015
977 11 25 2015
978 11 26 2015
979 11 27 2015
980 11 30 2015
981 12 1 2015
982 12 2 2015
983 12 3 2015
984 12 4 2015
985 12 7 2015
986 12 8 2015
987 12 9 2015
988 12 10 2015
989 12 11 2015
990 12 14 2015
991 12 15 2015
992 12 16 2015
993 12 17 2015
994 12 18 2015
995 12 21 2015
996 12 22 2015
997 12 23 2015
998 12 24 2015

[999 rows x 3 columns]

The output of the `datetime_mapper` is a dataframe containing the month, day and year columns which are aggregated at Date level.

## 3.5 `Myx` Class

Define `MyX` class that does nothing but passes the data the way it is to another class. This class will be used in feature union to preserve some columns without any change.

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class MyX(BaseEstimator, TransformerMixin):  

    def fit(self, X): 
      self.X_ = X
      return self
    
    def transform(self, X):
      return X

Create a `myx_mapper` that uses the `MyX` class to preserve the columns in `col_list` unchanged. The input is the aggregated data at `Date` level.

In [0]:
col_list = [col for col in add_to_train.columns if col not in ['Average','Date']] # get column names of the last version of train data except Date and Average
myx_mapper = DataFrameMapper(gen_features(columns=col_list,
                               classes=[{'class': MyX}]),
                          df_out=True)
myx_mapper.fit_transform(add_to_train)

Out[ 105 ]: 
 Status_Ballast Status_Laden Status_Port Country_Aland Islands \
0 197 285 217 0 
1 208 290 195 0 
2 204 279 191 0 
3 201 266 193 0 
4 203 276 209 0 
5 216 274 194 0 
6 206 266 199 0 
7 213 257 201 0 
8 203 256 234 0 
9 228 241 244 0 
10 225 238 241 0 
11 221 240 258 0 
12 225 256 250 0 
13 225 255 242 0 
14 229 266 227 0 
15 222 266 244 0 
16 232 268 246 0 
17 216 262 248 0 
18 233 264 230 0 
19 216 252 234 0 
20 209 254 239 0 
21 216 248 250 0 
22 214 241 270 0 
23 211 244 270 0 
24 199 253 262 0 
25 201 249 277 0 
26 209 245 281 0 
27 217 235 271 0 
28 211 246 253 0 
29 204 255 259 0 
.. ... ... ... ... 
969 341 414 354 0 
970 340 421 359 0 
971 337 434 347 0 
972 335 439 349 0 
973 337 410 361 0 
974 323 429 352 0 
975 332 421 353 0 
976 325 429 364 0 
977 328 424 369 0 
978 309 421 381 0 
979 310 438 383 0 
980 325 432 356 0 
981 322 433 365 0 
982 322 450 362 0 
983 310 432 374 0 
984 316 416 384 0 
985 334 418 378 0 
986 341 409 374 0 
987 325 411 379 0 
988 324 405 391 0 
989 300 413 404 0 
990 315 428 394 0 
991 315 432 356 0 
992 300 441 366 0 
993 307 439 376 0 
994 313 427 375 0 
995 310 441 342 0 
996 302 440 364 0 
997 302 447 362 0 
998 291 455 372 0 

 Country_Albania Country_Algeria Country_American Samoa Country_Angola \
0 0 0 0 0 
1 0 1 0 0 
2 0 2 0 0 
3 0 0 0 1 
4 0 1 0 2 
5 0 0 0 0 
6 0 2 0 4 
7 0 5 0 1 
8 0 3 0 1 
9 0 2 0 0 
10 0 1 0 2 
11 0 2 0 2 
12 0 0 0 3 
13 0 2 0 3 
14 0 0 0 1 
15 0 2 0 0 
16 0 3 0 2 
17 0 2 0 0 
18 0 0 0 1 
19 0 1 0 0 
20 0 1 0 1 
21 0 3 0 1 
22 0 1 0 0 
23 0 0 0 0 
24 0 1 0 0 
25 0 0 0 2 
26 0 0 0 1 
27 0 2 0 0 
28 0 3 0 1 
29 0 2 0 0 
.. ... ... ... ... 
969 0 3 0 1 
970 0 3 0 2 
971 0 3 1 0 
972 0 1 1 3 
973 0 1 2 2 
974 0 0 1 0 
975 0 2 0 3 
976 0 0 0 0 
977 0 2 0 2 
978 0 2 0 5 
979 0 1 0 1 
980 0 0 0 2 
981 0 2 0 2 
982 0 5 0 1 
983 0 3 0 1 
984 0 0 0 0 
985 0 2 0 1 
986 0 2 0 1 
987 0 0 0 1 
988 0 1 0 1 
989 0 3 0 2 
990 0 1 0 2 
991 0 3 0 1 
992 0 2 0 2 
993 0 0 0 1 
994 0 1 0 3 
995 0 6 0 0 
996 0 8 0 0 
997 0 2 0 1 
998 0 1 0 0 

 Country_Anguilla Country_Antigua and Barbuda ... \
0 3 5 ... 
1 0 3 ... 
2 0 1 ... 
3 0 1 ... 
4 1 2 ... 
5 0 2 ... 
6 1 1 ... 
7 0 2 ... 
8 2 0 ... 
9 1 6 ... 
10 0 7 ... 
11 1 4 ... 
12 1 3 ... 
13 1 2 ... 
14 0 1 ... 
15 0 1 ... 
16 0 1 ... 
17 2 1 ... 
18 0 2 ... 
19 0 2 ... 
20 0 2 ... 
21 2 1 ... 
22 3 2 ... 
23 0 2 ... 
24 2 1 ... 
25 0 4 ... 
26 1 3 ... 
27 0 2 ... 
28 0 2 ... 
29 0 5 ... 
.. ... ... ... 
969 0 3 ... 
970 0 1 ... 
971 2 0 ... 
972 0 2 ... 
973 2 2 ... 
974 0 2 ... 
975 0 3 ... 
976 3 3 ... 
977 0 4 ... 
978 1 3 ... 
979 1 3 ... 
980 1 2 ... 
981 1 2 ... 
982 1 0 ... 
983 2 1 ... 
984 0 4 ... 
985 0 4 ... 
986 1 4 ... 
987 0 3 ... 
988 2 0 ... 
989 1 1 ... 
990 1 3 ... 
991 1 4 ... 
992 2 4 ... 
993 1 3 ... 
994 0 5 ... 
995 1 1 ... 
996 0 0 ... 
997 0 1 ... 
998 1 0 ... 

 Country_Vanuatu Country_Venezuela Country_Vietnam \
0 0 0 9 
1 0 0 11 
2 0 0 4 
3 0 0 6 
4 0 0 12 
5 0 0 13 
6 0 0 7 
7 0 0 5 
8 0 0 11 
9 0 0 12 
10 0 0 12 
11 0 0 13 
12 0 0 16 
13 0 0 11 
14 0 0 19 
15 0 0 20 
16 0 0 16 
17 0 1 14 
18 0 0 11 
19 0 0 15 
20 0 0 16 
21 0 0 16 
22 0 0 12 
23 0 0 8 
24 0 0 27 
25 0 0 20 
26 0 0 14 
27 0 0 10 
28 0 0 16 
29 0 0 10 
.. ... ... ... 
969 1 0 8 
970 0 0 9 
971 0 0 10 
972 0 0 13 
973 0 0 8 
974 0 0 11 
975 0 0 10 
976 0 0 13 
977 0 0 11 
978 1 0 10 
979 1 0 15 
980 0 0 15 
981 0 0 11 
982 0 0 10 
983 0 0 14 
984 0 0 8 
985 0 0 12 
986 0 0 11 
987 0 0 8 
988 0 0 8 
989 0 0 7 
990 0 0 22 
991 0 0 12 
992 0 0 7 
993 0 0 8 
994 0 0 9 
995 0 0 7 
996 0 0 7 
997 0 0 16 
998 0 0 14 

 Country_Virgin Islands, British Country_Virgin Islands, U.S. \
0 0 0 
1 0 0 
2 0 0 
3 0 0 
4 0 0 
5 0 0 
6 1 0 
7 1 0 
8 0 0 
9 0 0 
10 1 0 
11 0 2 
12 1 0 
13 0 0 
14 0 1 
15 1 0 
16 2 0 
17 0 1 
18 0 0 
19 0 0 
20 0 0 
21 1 0 
22 0 0 
23 0 0 
24 0 0 
25 0 0 
26 0 1 
27 0 1 
28 0 0 
29 1 0 
.. ... ... 
969 0 0 
970 1 1 
971 0 0 
972 0 0 
973 0 1 
974 1 0 
975 0 0 
976 0 1 
977 1 1 
978 0 0 
979 1 0 
980 2 1 
981 0 0 
982

The output is the same as the input. It returns the columns given in `col_list` the way they are.

Create a `feature_union` that merges the output of `Lags`, `DatetimeGenerator` and `MyX` classes.

In [0]:
feature_union = sklearn.pipeline.FeatureUnion([('lag features',    lag_mapper),
                                              ('date time features',    datetime_mapper),
                                               ('myx', myx_mapper)
                                              ])    
feature_union_output = feature_union.fit_transform(add_to_train)
feature_union_output, feature_union_output.shape

Out[ 106 ]: 
(array([[nan, nan, nan, ..., 2, 181206, 0],
 [24795.0, nan, nan, ..., 1, 181205, '0.25253'],
 [22697.0, 24795.0, nan, ..., 1, 181357, '-1.36601'],
 ...,
 [5156.0, 5190.0, 5008.0, ..., 0, 183583, '4.54414'],
 [4958.0, 5156.0, 5190.0, ..., 3, 183739, '1.77187'],
 [4897.0, 4958.0, 5156.0, ..., 3, 183821, '2.3395']], dtype=object),
 (999, 188))

The output of `feature_union` is the concatenated output of `lag_mapper`, `datetime_mapper` and `myx_mapper`.

## 3.6 `MakeDataframe` Class

Define `MakeDataframe` class. This class converts the numpy array output of `feature union` to a pandas dataframe and names the columns by the list given in`col_names` parameter.

In [0]:
class MakeDataframe(BaseEstimator, TransformerMixin):
  def __init__(self, col_names):
    self.col_names = col_names
  
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.name_cols(self.make_df(X))
    return self.transformed_
  
  def make_df(self, X):
    X = pd.DataFrame(X)
    return X
  
  def name_cols(self, X):
    X.columns = self.col_names
    return X

Generate an object of `MakeDataframe` class and check the output of its transform method. The input is the output of feature union.

In [0]:
col_names=['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'month', 'day', 'year'] + col_list # column names to be given to columns of feature union output
make_dataframe = MakeDataframe(col_names)
make_dataframe_output= make_dataframe.fit_transform(feature_union_output)
make_dataframe_output.shape, make_dataframe_output

Out[ 108 ]: 
((999, 188),
 lag_1 lag_2 lag_3 lag_4 lag_5 month day year Status_Ballast \
 0 NaN NaN NaN NaN NaN 1 3 2012 197 
 1 24795 NaN NaN NaN NaN 1 4 2012 208 
 2 22697 24795 NaN NaN NaN 1 5 2012 204 
 3 18729 22697 24795 NaN NaN 1 6 2012 201 
 4 16424 18729 22697 24795 NaN 1 9 2012 203 
 5 15498 16424 18729 22697 24795 1 10 2012 216 
 6 14368 15498 16424 18729 22697 1 11 2012 206 
 7 13129 14368 15498 16424 18729 1 12 2012 213 
 8 10999 13129 14368 15498 16424 1 13 2012 203 
 9 10170 10999 13129 14368 15498 1 16 2012 228 
 10 9562 10170 10999 13129 14368 1 17 2012 225 
 11 8869 9562 10170 10999 13129 1 18 2012 221 
 12 8191 8869 9562 10170 10999 1 19 2012 225 
 13 7898 8191 8869 9562 10170 1 20 2012 225 
 14 7783 7898 8191 8869 9562 1 23 2012 229 
 15 7726 7783 7898 8191 8869 1 24 2012 222 
 16 7239 7726 7783 7898 8191 1 25 2012 232 
 17 7074 7239 7726 7783 7898 1 26 2012 216 
 18 6843 7074 7239 7726 7783 1 27 2012 233 
 19 6679 6843 7074 7239 7726 1 30 2012 216 
 20 6605 6679 6843 7074 7239 1 31 2012 209 
 21 6496 6605 6679 6843 7074 2 1 2012 216 
 22 6444 6496 6605 6679 6843 2 2 2012 214 
 23 6430 6444 6496 6605 6679 2 3 2012 211 
 24 6370 6430 6444 6496 6605 2 6 2012 199 
 25 6333 6370 6430 6444 6496 2 7 2012 201 
 26 6362 6333 6370 6430 6444 2 8 2012 209 
 27 6352 6362 6333 6370 6430 2 9 2012 217 
 28 6360 6352 6362 6333 6370 2 10 2012 211 
 29 6346 6360 6352 6362 6333 2 13 2012 204 
 .. ... ... ... ... ... ... .. ... ... 
 969 6655 7062 7543 7518 7358 11 13 2015 341 
 970 6280 6655 7062 7543 7518 11 16 2015 340 
 971 6135 6280 6655 7062 7543 11 17 2015 337 
 972 5968 6135 6280 6655 7062 11 18 2015 335 
 973 5597 5968 6135 6280 6655 11 19 2015 337 
 974 5280 5597 5968 6135 6280 11 20 2015 323 
 975 5211 5280 5597 5968 6135 11 23 2015 332 
 976 5967 5211 5280 5597 5968 11 24 2015 325 
 977 6409 5967 5211 5280 5597 11 25 2015 328 
 978 7062 6409 5967 5211 5280 11 26 2015 309 
 979 7682 7062 6409 5967 5211 11 27 2015 310 
 980 8396 7682 7062 6409 5967 11 30 2015 325 
 981 8486 8396 7682 7062 6409 12 1 2015 322 
 982 8977 8486 8396 7682 7062 12 2 2015 322 
 983 8715 8977 8486 8396 7682 12 3 2015 310 
 984 8189 8715 8977 8486 8396 12 4 2015 316 
 985 7863 8189 8715 8977 8486 12 7 2015 334 
 986 7522 7863 8189 8715 8977 12 8 2015 341 
 987 7609 7522 7863 8189 8715 12 9 2015 325 
 988 7486 7609 7522 7863 8189 12 10 2015 324 
 989 7181 7486 7609 7522 7863 12 11 2015 300 
 990 6799 7181 7486 7609 7522 12 14 2015 315 
 991 6341 6799 7181 7486 7609 12 15 2015 315 
 992 5514 6341 6799 7181 7486 12 16 2015 300 
 993 5044 5514 6341 6799 7181 12 17 2015 307 
 994 5008 5044 5514 6341 6799 12 18 2015 313 
 995 5190 5008 5044 5514 6341 12 21 2015 310 
 996 5156 5190 5008 5044 5514 12 22 2015 302 
 997 4958 5156 5190 5008 5044 12 23 2015 302 
 998 4897 4958 5156 5190 5008 12 24 2015 291 
 
 Status_Laden ... Country_Vanuatu Country_Venezuela \
 0 285 ... 0 0 
 1 290 ... 0 0 
 2 279 ... 0 0 
 3 266 ... 0 0 
 4 276 ... 0 0 
 5 274 ... 0 0 
 6 266 ... 0 0 
 7 257 ... 0 0 
 8 256 ... 0 0 
 9 241 ... 0 0 
 10 238 ... 0 0 
 11 240 ... 0 0 
 12 256 ... 0 0 
 13 255 ... 0 0 
 14 266 ... 0 0 
 15 266 ... 0 0 
 16 268 ... 0 0 
 17 262 ... 0 1 
 18 264 ... 0 0 
 19 252 ... 0 0 
 20 254 ... 0 0 
 21 248 ... 0 0 
 22 241 ... 0 0 
 23 244 ... 0 0 
 24 253 ... 0 0 
 25 249 ... 0 0 
 26 245 ... 0 0 
 27 235 ... 0 0 
 28 246 ... 0 0 
 29 255 ... 0 0 
 .. ... ... ... ... 
 969 414 ... 1 0 
 970 421 ... 0 0 
 971 434 ... 0 0 
 972 439 ... 0 0 
 973 410 ... 0 0 
 974 429 ... 0 0 
 975 421 ... 0 0 
 976 429 ... 0 0 
 977 424 ... 0 0 
 978 421 ... 1 0 
 979 438 ... 1 0 
 980 432 ... 0 0 
 981 433 ... 0 0 
 982 450 ... 0 0 
 983 432 ... 0 0 
 984 416 ... 0 0 
 985 418 ... 0 0 
 986 409 ... 0 0 
 987 411 ... 0 0 
 988 405 ... 0 0 
 989 413 ... 0 0 
 990 428 ... 0 0 
 991 432 ... 0 0 
 992 441 ... 0 0 
 993 439 ... 0 0 
 994 427 ... 0 0 
 995 441 ... 0 0 
 996 440 ... 0 0 
 997 447 ... 0 0 
 998 455 ... 0 0 
 
 Country_Vietnam Country_Virgin Islands, British \


The output of the `MakeDataframe` object is a pandas dataframe with column names as given in `col_names` parameter.

## 3.7 `WindowFeature` Class

Define the `WindowFeature` class. This class will use the output of `MakeDataframe` class and will provide rolling window calculations. The `window_width` parameter is the size of the moving window. This is the number of observations used for calculating the statistic. Each window will be a fixed size. The `rolling()` function performs window calcualtions and creates a new data structure with the window of values at each time step. Here we use `mean` as aggregate function.

In [0]:
class WindowFeature(BaseEstimator, TransformerMixin):
  
  def __init__(self, window_width):
    self.window_width = window_width
    
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.create_window(X)
    return self.transformed_
  
  def create_window(self, X):
    filter_col = [col for col in X if col.startswith('lag')]
    window_df = X.loc[:,filter_col]
    window = window_df.rolling(window=self.window_width) 
    means =  window.mean()
    columns = [str(i) for i in [i +1 for i in range(means.shape[1])]]
    means.columns = ['mean_' + i for i in columns]
    X = pd.concat([X, means], axis=1)
    return X

Generate an object of `WindowFeature` class and check the output of its transform method. The input is the output of `MakeDataframe` class.

In [0]:
window_generator = WindowFeature(window_width)
window_generator_output = window_generator.fit_transform(make_dataframe_output)
window_generator_output.shape, window_generator_output 

Out[ 110 ]: 
((999, 193),
 lag_1 lag_2 lag_3 lag_4 lag_5 month day year Status_Ballast \
 0 NaN NaN NaN NaN NaN 1 3 2012 197 
 1 24795 NaN NaN NaN NaN 1 4 2012 208 
 2 22697 24795 NaN NaN NaN 1 5 2012 204 
 3 18729 22697 24795 NaN NaN 1 6 2012 201 
 4 16424 18729 22697 24795 NaN 1 9 2012 203 
 5 15498 16424 18729 22697 24795 1 10 2012 216 
 6 14368 15498 16424 18729 22697 1 11 2012 206 
 7 13129 14368 15498 16424 18729 1 12 2012 213 
 8 10999 13129 14368 15498 16424 1 13 2012 203 
 9 10170 10999 13129 14368 15498 1 16 2012 228 
 10 9562 10170 10999 13129 14368 1 17 2012 225 
 11 8869 9562 10170 10999 13129 1 18 2012 221 
 12 8191 8869 9562 10170 10999 1 19 2012 225 
 13 7898 8191 8869 9562 10170 1 20 2012 225 
 14 7783 7898 8191 8869 9562 1 23 2012 229 
 15 7726 7783 7898 8191 8869 1 24 2012 222 
 16 7239 7726 7783 7898 8191 1 25 2012 232 
 17 7074 7239 7726 7783 7898 1 26 2012 216 
 18 6843 7074 7239 7726 7783 1 27 2012 233 
 19 6679 6843 7074 7239 7726 1 30 2012 216 
 20 6605 6679 6843 7074 7239 1 31 2012 209 
 21 6496 6605 6679 6843 7074 2 1 2012 216 
 22 6444 6496 6605 6679 6843 2 2 2012 214 
 23 6430 6444 6496 6605 6679 2 3 2012 211 
 24 6370 6430 6444 6496 6605 2 6 2012 199 
 25 6333 6370 6430 6444 6496 2 7 2012 201 
 26 6362 6333 6370 6430 6444 2 8 2012 209 
 27 6352 6362 6333 6370 6430 2 9 2012 217 
 28 6360 6352 6362 6333 6370 2 10 2012 211 
 29 6346 6360 6352 6362 6333 2 13 2012 204 
 .. ... ... ... ... ... ... .. ... ... 
 969 6655 7062 7543 7518 7358 11 13 2015 341 
 970 6280 6655 7062 7543 7518 11 16 2015 340 
 971 6135 6280 6655 7062 7543 11 17 2015 337 
 972 5968 6135 6280 6655 7062 11 18 2015 335 
 973 5597 5968 6135 6280 6655 11 19 2015 337 
 974 5280 5597 5968 6135 6280 11 20 2015 323 
 975 5211 5280 5597 5968 6135 11 23 2015 332 
 976 5967 5211 5280 5597 5968 11 24 2015 325 
 977 6409 5967 5211 5280 5597 11 25 2015 328 
 978 7062 6409 5967 5211 5280 11 26 2015 309 
 979 7682 7062 6409 5967 5211 11 27 2015 310 
 980 8396 7682 7062 6409 5967 11 30 2015 325 
 981 8486 8396 7682 7062 6409 12 1 2015 322 
 982 8977 8486 8396 7682 7062 12 2 2015 322 
 983 8715 8977 8486 8396 7682 12 3 2015 310 
 984 8189 8715 8977 8486 8396 12 4 2015 316 
 985 7863 8189 8715 8977 8486 12 7 2015 334 
 986 7522 7863 8189 8715 8977 12 8 2015 341 
 987 7609 7522 7863 8189 8715 12 9 2015 325 
 988 7486 7609 7522 7863 8189 12 10 2015 324 
 989 7181 7486 7609 7522 7863 12 11 2015 300 
 990 6799 7181 7486 7609 7522 12 14 2015 315 
 991 6341 6799 7181 7486 7609 12 15 2015 315 
 992 5514 6341 6799 7181 7486 12 16 2015 300 
 993 5044 5514 6341 6799 7181 12 17 2015 307 
 994 5008 5044 5514 6341 6799 12 18 2015 313 
 995 5190 5008 5044 5514 6341 12 21 2015 310 
 996 5156 5190 5008 5044 5514 12 22 2015 302 
 997 4958 5156 5190 5008 5044 12 23 2015 302 
 998 4897 4958 5156 5190 5008 12 24 2015 291 
 
 Status_Laden ... Country_Wallis and Futuna Country_Western Sahara \
 0 285 ... 0 0 
 1 290 ... 0 1 
 2 279 ... 0 2 
 3 266 ... 0 2 
 4 276 ... 0 1 
 5 274 ... 0 3 
 6 266 ... 0 1 
 7 257 ... 0 1 
 8 256 ... 0 0 
 9 241 ... 0 0 
 10 238 ... 0 1 
 11 240 ... 0 2 
 12 256 ... 0 0 
 13 255 ... 0 0 
 14 266 ... 0 1 
 15 266 ... 0 0 
 16 268 ... 0 2 
 17 262 ... 0 0 
 18 264 ... 0 3 
 19 252 ... 0 1 
 20 254 ... 0 1 
 21 248 ... 0 1 
 22 241 ... 0 2 
 23 244 ... 0 2 
 24 253 ... 0 1 
 25 249 ... 0 1 
 26 245 ... 0 2 
 27 235 ... 0 3 
 28 246 ... 0 3 
 29 255 ... 0 2 
 .. ... ... ... ... 
 969 414 ... 0 2 
 970 421 ... 0 0 
 971 434 ... 0 0 
 972 439 ... 0 3 
 973 410 ... 0 0 
 974 429 ... 0 0 
 975 421 ... 0 1 
 976 429 ... 0 1 
 977 424 ... 0 1 
 978 421 ... 0 1 
 979 438 ... 0 0 
 980 432 ... 0 1 
 981 433 ... 0 0 
 982 450 ... 0 0 
 983 432 ... 0 2 
 984 416 ... 0 0 
 985 418 ... 0 2 
 986 409 ... 0 0 
 987 411 ... 0 1 
 988 405 ... 0 1 
 989 413 ... 0 0 
 990 428 ... 0 0 
 991 432 ... 0 3 
 992 441 ... 0 3 
 993 439 ... 0 3 
 994 427 ... 0 1 
 995 441 ... 0 3 
 996 440 ... 0 1 
 997 447 ... 0 2 
 998 455 ... 0 3 
 
 Country_Yemen dwt DCOILWTICO_PCH me

The output of the above `WindowFeature` object is a dataframe containing the windows generated from the lag variables. There are five window features generated. There are NA values because the past values at those points do not exist.

## 3.8 `RemoveNa` Class

Define the `RemoveNa` class . This class drops the rows with the NA values created from the `Lags` class and the `WindowFeature` class.

In [0]:
class RemoveNa(BaseEstimator, TransformerMixin):
  
  def __init__(self):
    self
  
  def fit(self, X, y=None):
    return self
    
  def transform(self, X):
    self.transformed_ = self.remove_na(X)
    return self.transformed_
  
  def remove_na(self, X):
    X = X.dropna(axis=0, how='any')
    return X

Generate an object of `RemoveNa` class and check the output of its transform method. The input is the output of `WindowFeature` class.

In [0]:
remove_na = RemoveNa()
remove_na_output = remove_na.fit_transform(window_generator_output)
remove_na_output.shape, remove_na_output

Out[ 112 ]: 
((993, 193),
 lag_1 lag_2 lag_3 lag_4 lag_5 month day year Status_Ballast \
 6 14368 15498 16424 18729 22697 1 11 2012 206 
 7 13129 14368 15498 16424 18729 1 12 2012 213 
 8 10999 13129 14368 15498 16424 1 13 2012 203 
 9 10170 10999 13129 14368 15498 1 16 2012 228 
 10 9562 10170 10999 13129 14368 1 17 2012 225 
 11 8869 9562 10170 10999 13129 1 18 2012 221 
 12 8191 8869 9562 10170 10999 1 19 2012 225 
 13 7898 8191 8869 9562 10170 1 20 2012 225 
 14 7783 7898 8191 8869 9562 1 23 2012 229 
 15 7726 7783 7898 8191 8869 1 24 2012 222 
 16 7239 7726 7783 7898 8191 1 25 2012 232 
 17 7074 7239 7726 7783 7898 1 26 2012 216 
 18 6843 7074 7239 7726 7783 1 27 2012 233 
 19 6679 6843 7074 7239 7726 1 30 2012 216 
 20 6605 6679 6843 7074 7239 1 31 2012 209 
 21 6496 6605 6679 6843 7074 2 1 2012 216 
 22 6444 6496 6605 6679 6843 2 2 2012 214 
 23 6430 6444 6496 6605 6679 2 3 2012 211 
 24 6370 6430 6444 6496 6605 2 6 2012 199 
 25 6333 6370 6430 6444 6496 2 7 2012 201 
 26 6362 6333 6370 6430 6444 2 8 2012 209 
 27 6352 6362 6333 6370 6430 2 9 2012 217 
 28 6360 6352 6362 6333 6370 2 10 2012 211 
 29 6346 6360 6352 6362 6333 2 13 2012 204 
 30 6426 6346 6360 6352 6362 2 14 2012 204 
 31 6457 6426 6346 6360 6352 2 15 2012 208 
 32 6452 6457 6426 6346 6360 2 16 2012 207 
 33 6434 6452 6457 6426 6346 2 17 2012 213 
 34 6404 6434 6452 6457 6426 2 20 2012 234 
 35 6362 6404 6434 6452 6457 2 21 2012 219 
 .. ... ... ... ... ... ... .. ... ... 
 969 6655 7062 7543 7518 7358 11 13 2015 341 
 970 6280 6655 7062 7543 7518 11 16 2015 340 
 971 6135 6280 6655 7062 7543 11 17 2015 337 
 972 5968 6135 6280 6655 7062 11 18 2015 335 
 973 5597 5968 6135 6280 6655 11 19 2015 337 
 974 5280 5597 5968 6135 6280 11 20 2015 323 
 975 5211 5280 5597 5968 6135 11 23 2015 332 
 976 5967 5211 5280 5597 5968 11 24 2015 325 
 977 6409 5967 5211 5280 5597 11 25 2015 328 
 978 7062 6409 5967 5211 5280 11 26 2015 309 
 979 7682 7062 6409 5967 5211 11 27 2015 310 
 980 8396 7682 7062 6409 5967 11 30 2015 325 
 981 8486 8396 7682 7062 6409 12 1 2015 322 
 982 8977 8486 8396 7682 7062 12 2 2015 322 
 983 8715 8977 8486 8396 7682 12 3 2015 310 
 984 8189 8715 8977 8486 8396 12 4 2015 316 
 985 7863 8189 8715 8977 8486 12 7 2015 334 
 986 7522 7863 8189 8715 8977 12 8 2015 341 
 987 7609 7522 7863 8189 8715 12 9 2015 325 
 988 7486 7609 7522 7863 8189 12 10 2015 324 
 989 7181 7486 7609 7522 7863 12 11 2015 300 
 990 6799 7181 7486 7609 7522 12 14 2015 315 
 991 6341 6799 7181 7486 7609 12 15 2015 315 
 992 5514 6341 6799 7181 7486 12 16 2015 300 
 993 5044 5514 6341 6799 7181 12 17 2015 307 
 994 5008 5044 5514 6341 6799 12 18 2015 313 
 995 5190 5008 5044 5514 6341 12 21 2015 310 
 996 5156 5190 5008 5044 5514 12 22 2015 302 
 997 4958 5156 5190 5008 5044 12 23 2015 302 
 998 4897 4958 5156 5190 5008 12 24 2015 291 
 
 Status_Laden ... Country_Wallis and Futuna Country_Western Sahara \
 6 266 ... 0 1 
 7 257 ... 0 1 
 8 256 ... 0 0 
 9 241 ... 0 0 
 10 238 ... 0 1 
 11 240 ... 0 2 
 12 256 ... 0 0 
 13 255 ... 0 0 
 14 266 ... 0 1 
 15 266 ... 0 0 
 16 268 ... 0 2 
 17 262 ... 0 0 
 18 264 ... 0 3 
 19 252 ... 0 1 
 20 254 ... 0 1 
 21 248 ... 0 1 
 22 241 ... 0 2 
 23 244 ... 0 2 
 24 253 ... 0 1 
 25 249 ... 0 1 
 26 245 ... 0 2 
 27 235 ... 0 3 
 28 246 ... 0 3 
 29 255 ... 0 2 
 30 254 ... 0 2 
 31 260 ... 0 2 
 32 249 ... 0 3 
 33 243 ... 0 2 
 34 250 ... 0 3 
 35 238 ... 0 1 
 .. ... ... ... ... 
 969 414 ... 0 2 
 970 421 ... 0 0 
 971 434 ... 0 0 
 972 439 ... 0 3 
 973 410 ... 0 0 
 974 429 ... 0 0 
 975 421 ... 0 1 
 976 429 ... 0 1 
 977 424 ... 0 1 
 978 421 ... 0 1 
 979 438 ... 0 0 
 980 432 ... 0 1 
 981 433 ... 0 0 
 982 450 ... 0 0 
 983 432 ... 0 2 
 984 416 ... 0 0 
 985 418 ... 0 2 
 986 409 ... 0 0 
 987 411 ... 0 1 
 988 405 ... 0 1 
 989 413 ... 0 0 
 990 428 ... 0 0 
 991 432 ... 0 3 
 992 441 ... 0 3 
 993 439 ... 0 3 
 994 427 ... 0 1 
 995 441 ... 0 3 
 996 440 ... 0 1 
 997 447 ... 0 2 
 998 455 ... 0 3 
 
 Country_Yemen dwt 

The output of the above `RemoveNa` object is a dataframe that does not contain any row with NA values.

## 3.9 `ResetIndex` Class

Define `ResetIndex` class. This class resets the index and removes the index column from the dataset. This class will be used before the estimator gets applied in pipeline.

In [0]:
class ResetIndex(BaseEstimator, TransformerMixin):
  
  def __init__(self):
    self
  
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    self.transformed_ = self.prepare(X)
    return self.transformed_
  
  def prepare(self, X):
    X = X.reset_index()
    X = X.loc[:, X.columns != 'index']
    return X

Generate an object of `ResetIndex` class and check the output of its transform method.

In [0]:
reset_index = ResetIndex()
X_train_final = reset_index.fit_transform(remove_na_output)
X_train_final.shape, X_train_final

Out[ 114 ]: 
((993, 193),
 lag_1 lag_2 lag_3 lag_4 lag_5 month day year Status_Ballast \
 0 14368 15498 16424 18729 22697 1 11 2012 206 
 1 13129 14368 15498 16424 18729 1 12 2012 213 
 2 10999 13129 14368 15498 16424 1 13 2012 203 
 3 10170 10999 13129 14368 15498 1 16 2012 228 
 4 9562 10170 10999 13129 14368 1 17 2012 225 
 5 8869 9562 10170 10999 13129 1 18 2012 221 
 6 8191 8869 9562 10170 10999 1 19 2012 225 
 7 7898 8191 8869 9562 10170 1 20 2012 225 
 8 7783 7898 8191 8869 9562 1 23 2012 229 
 9 7726 7783 7898 8191 8869 1 24 2012 222 
 10 7239 7726 7783 7898 8191 1 25 2012 232 
 11 7074 7239 7726 7783 7898 1 26 2012 216 
 12 6843 7074 7239 7726 7783 1 27 2012 233 
 13 6679 6843 7074 7239 7726 1 30 2012 216 
 14 6605 6679 6843 7074 7239 1 31 2012 209 
 15 6496 6605 6679 6843 7074 2 1 2012 216 
 16 6444 6496 6605 6679 6843 2 2 2012 214 
 17 6430 6444 6496 6605 6679 2 3 2012 211 
 18 6370 6430 6444 6496 6605 2 6 2012 199 
 19 6333 6370 6430 6444 6496 2 7 2012 201 
 20 6362 6333 6370 6430 6444 2 8 2012 209 
 21 6352 6362 6333 6370 6430 2 9 2012 217 
 22 6360 6352 6362 6333 6370 2 10 2012 211 
 23 6346 6360 6352 6362 6333 2 13 2012 204 
 24 6426 6346 6360 6352 6362 2 14 2012 204 
 25 6457 6426 6346 6360 6352 2 15 2012 208 
 26 6452 6457 6426 6346 6360 2 16 2012 207 
 27 6434 6452 6457 6426 6346 2 17 2012 213 
 28 6404 6434 6452 6457 6426 2 20 2012 234 
 29 6362 6404 6434 6452 6457 2 21 2012 219 
 .. ... ... ... ... ... ... .. ... ... 
 963 6655 7062 7543 7518 7358 11 13 2015 341 
 964 6280 6655 7062 7543 7518 11 16 2015 340 
 965 6135 6280 6655 7062 7543 11 17 2015 337 
 966 5968 6135 6280 6655 7062 11 18 2015 335 
 967 5597 5968 6135 6280 6655 11 19 2015 337 
 968 5280 5597 5968 6135 6280 11 20 2015 323 
 969 5211 5280 5597 5968 6135 11 23 2015 332 
 970 5967 5211 5280 5597 5968 11 24 2015 325 
 971 6409 5967 5211 5280 5597 11 25 2015 328 
 972 7062 6409 5967 5211 5280 11 26 2015 309 
 973 7682 7062 6409 5967 5211 11 27 2015 310 
 974 8396 7682 7062 6409 5967 11 30 2015 325 
 975 8486 8396 7682 7062 6409 12 1 2015 322 
 976 8977 8486 8396 7682 7062 12 2 2015 322 
 977 8715 8977 8486 8396 7682 12 3 2015 310 
 978 8189 8715 8977 8486 8396 12 4 2015 316 
 979 7863 8189 8715 8977 8486 12 7 2015 334 
 980 7522 7863 8189 8715 8977 12 8 2015 341 
 981 7609 7522 7863 8189 8715 12 9 2015 325 
 982 7486 7609 7522 7863 8189 12 10 2015 324 
 983 7181 7486 7609 7522 7863 12 11 2015 300 
 984 6799 7181 7486 7609 7522 12 14 2015 315 
 985 6341 6799 7181 7486 7609 12 15 2015 315 
 986 5514 6341 6799 7181 7486 12 16 2015 300 
 987 5044 5514 6341 6799 7181 12 17 2015 307 
 988 5008 5044 5514 6341 6799 12 18 2015 313 
 989 5190 5008 5044 5514 6341 12 21 2015 310 
 990 5156 5190 5008 5044 5514 12 22 2015 302 
 991 4958 5156 5190 5008 5044 12 23 2015 302 
 992 4897 4958 5156 5190 5008 12 24 2015 291 
 
 Status_Laden ... Country_Wallis and Futuna Country_Western Sahara \
 0 266 ... 0 1 
 1 257 ... 0 1 
 2 256 ... 0 0 
 3 241 ... 0 0 
 4 238 ... 0 1 
 5 240 ... 0 2 
 6 256 ... 0 0 
 7 255 ... 0 0 
 8 266 ... 0 1 
 9 266 ... 0 0 
 10 268 ... 0 2 
 11 262 ... 0 0 
 12 264 ... 0 3 
 13 252 ... 0 1 
 14 254 ... 0 1 
 15 248 ... 0 1 
 16 241 ... 0 2 
 17 244 ... 0 2 
 18 253 ... 0 1 
 19 249 ... 0 1 
 20 245 ... 0 2 
 21 235 ... 0 3 
 22 246 ... 0 3 
 23 255 ... 0 2 
 24 254 ... 0 2 
 25 260 ... 0 2 
 26 249 ... 0 3 
 27 243 ... 0 2 
 28 250 ... 0 3 
 29 238 ... 0 1 
 .. ... ... ... ... 
 963 414 ... 0 2 
 964 421 ... 0 0 
 965 434 ... 0 0 
 966 439 ... 0 3 
 967 410 ... 0 0 
 968 429 ... 0 0 
 969 421 ... 0 1 
 970 429 ... 0 1 
 971 424 ... 0 1 
 972 421 ... 0 1 
 973 438 ... 0 0 
 974 432 ... 0 1 
 975 433 ... 0 0 
 976 450 ... 0 0 
 977 432 ... 0 2 
 978 416 ... 0 0 
 979 418 ... 0 2 
 980 409 ... 0 0 
 981 411 ... 0 1 
 982 405 ... 0 1 
 983 413 ... 0 0 
 984 428 ... 0 0 
 985 432 ... 0 3 
 986 441 ... 0 3 
 987 439 ... 0 3 
 988 427 ... 0 1 
 989 441 ... 0 3 
 990 440 ... 0 1 
 991 447 ... 0 2 
 992 455 ... 0 3 
 
 Country_Yemen dwt DCOILWTICO_P

The output of the above `ResetIndex` object is a final version of train data. It is a dataframe with 993 rows and 193 columns. The index is reset to start from zero.

# 4. Conclusion

In this notebook we defined 9 classes that will process the data and put in the desired format. The first class will add location features and aggregate the data to date level. The `dwt` value is the mean of the dead weights at that date while the location variables contain information about the number of ships at that location on that particular date. The status variables show the number of ships in each status condition per date. The `ExternalFeatures` class adds the daily crude oil price per date. The `Lags` class generates lags of `Average` column and `DatetimeGenerator` class creates month, day and year columns from the `Date` column. `MyX` class passes columns untouched. Then, we defined a feature union that concatenates the output of `lag_mapper`, `datetime_mapper` and `myx_mapper`. The output of which is used as input in the `MakeDataframe` class. This class will put the numpy array in a dataframe and assign column names. The output of `MakeDataframe` goes to `WindowFeature` class which generates mean columns out of the lags using the moving average window. The last two classes `RemoveNa` and `ResetIndex` removes the NA values and reset the index to start from zero. This is the data that will be used in other notebooks to train the models.